# Plan of action

Here I'll be working towards developing a script for automated data extraction and Kallisto processing.

The workflow to develop:
- Extract metadata
- From metadata, identify SRR accessions (I can't remember what they were called)
- I need a Kallisto index. I'm tempted to just download the prebuilt indices...
- Download the FASTQ files
- Perform Kallisto processing.

The goal is the have a bash script that can take in a single GEO accession and do all this. 

In [9]:
Rscript ./RScript_GetGEOMetadata.r -g GSE273561 -o ../Testing/GSE273561

Output directory exists: ../Testing/GSE273561 
Retrieving metadata for GEO accession: GSE273561 
Found 2 file(s)
GSE273561-GPL24247_series_matrix.txt.gz
Annotation GPL not available, so will use submitter GPL instead
GSE273561-GPL24676_series_matrix.txt.gz
Annotation GPL not available, so will use submitter GPL instead
Saving metadata to: ../Testing/GSE273561/GSE273561-GPL24247_series_matrix_metadata.csv 
Saving metadata to: ../Testing/GSE273561/GSE273561-GPL24676_series_matrix_metadata.csv 
Metadata saved successfully!



Hypothetically (and soon to be reality), the output of the above... might possibly be usable...?

What I REALLY need is the SRA IDs. I can see in these datasets the information is in "relations," but I am willing to bet that is not the case for all datasets. So my options are:
a) Use an LLM to find this information in the metadata
b) Optimize the original script to extract that information

I will go with option A until I receive alternative advice...

Alternative advice received and sent by me - thanks me. I do think an LLM to check if the metadata and extracted SRR IDs (from below) align - this can form PART of the evaluation, in conjunction with interpreting the std.err/std.out.

Update - going from I "think" to "I can make a strong case for." For the Kallisto processing, I need to correctly identify the species. Text-matching is completely possible, but I would honestly feel more confident with an LLM to back this up. 

In [6]:
esearch -db gds -query "GSE273561[ACCN] AND GSM[ETYP]" | \
efetch -format docsum | \
xtract -pattern ExtRelation -element RelationType,TargetObject | \
cut -f2 > output.txt

tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified


The output of above is inconvenient, I do think it's causing the code to hang (there are issues with =TERM=). Nonetheless, the output.txt is what I expect. I can now link these to SRA IDs (I hope).

Wait, the above isn't hanging now? Not complaining... but something I need to keep in mind.

In [ ]:
esearch -db sra -query "SRX25627130" | efetch -format docsum | xtract -pattern DocumentSummary -element Run@acc

myuser@e3fa3416bfeb:~/work/Code/2_extract_data$ ^C
myuser@e3fa3416bfeb:~/work/Code/2_extract_data$ tput: tput: No value for $TERM and no -T specifiedNo value for $TERM and no -T specified

tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified


In [12]:
echo '#!/bin/bash' > my_script.sh
echo '
# Initialize the output file with headers
echo -e "experiment\tSRA_ID" > results.txt

# Use a for loop to iterate over each line in output.txt
for experiment in $(cat output.txt); do
  # Print a message indicating the current experiment being processed
  echo "Processing experiment: $experiment"

  # Run the esearch and efetch commands to get the corresponding SRA ID
  sra_id=$(esearch -db sra -query "$experiment" | efetch -format docsum | xtract -pattern DocumentSummary -element Run@acc)

  # If no SRA ID is found, set sra_id to "No SRA ID found"
  if [ -z "$sra_id" ]; then
    sra_id="No SRA ID found"
  fi

  # Append the experiment and SRA ID to the results.txt file
  echo -e "$experiment\t$sra_id" >> results.txt
done
' >> my_script.sh
chmod +x my_script.sh  # Make the script executable

In [13]:
cat my_script.sh

#!/bin/bash

# Initialize the output file with headers
echo -e "experiment\tSRA_ID" > results.txt

# Use a for loop to iterate over each line in output.txt
for experiment in $(cat output.txt); do
  # Print a message indicating the current experiment being processed
  echo "Processing experiment: $experiment"

  # Run the esearch and efetch commands to get the corresponding SRA ID
  sra_id=$(esearch -db sra -query "$experiment" | efetch -format docsum | xtract -pattern DocumentSummary -element Run@acc)

  # If no SRA ID is found, set sra_id to "No SRA ID found"
  if [ -z "$sra_id" ]; then
    sra_id="No SRA ID found"
  fi

  # Append the experiment and SRA ID to the results.txt file
  echo -e "$experiment\t$sra_id" >> results.txt
done



In [14]:
./my_script.sh

Processing experiment: SRX25627136
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
Processing experiment: SRX25627135
tput: tput: No value for $TERM

The last thing I'll do before I head off for the day is to check if I am able to download FASTQ files at all on this...

In [18]:
prefetch SRR30159865

2024-08-12T08:27:10 prefetch.3.1.1: 1) Resolving 'SRR30159865'...
2024-08-12T08:27:12 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-08-12T08:27:13 prefetch.3.1.1: 1) Downloading 'SRR30159865'...
2024-08-12T08:27:13 prefetch.3.1.1:  SRA Normalized Format file is being retrieved
2024-08-12T08:27:13 prefetch.3.1.1:  Downloading via HTTPS...
2024-08-12T08:31:49 prefetch.3.1.1:  HTTPS download succeed
2024-08-12T08:32:07 prefetch.3.1.1:  'SRR30159865' is valid: 2192138475 bytes were streamed from 2192130471
2024-08-12T08:32:07 prefetch.3.1.1: 1) 'SRR30159865' was downloaded successfully
2024-08-12T08:32:07 prefetch.3.1.1: 'SRR30159865' has 0 unresolved dependencies


In [19]:
fasterq-dump SRR30159865

spots read      : 23,539,692
reads read      : 47,079,384
reads written   : 47,079,384


In [6]:
cd ~/work/notebooks/2_extract_data
./get_sra_ids.sh GSE268034

Processing GEO accession: GSE268034
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
tput: No value for $TERM and no -T specified
Processing sample: GSM8284513
tput: No value for $TERM and no -T

In [5]:
echo $PATH

/usr/local/bin:/home/myuser/.cache/pypoetry/virtualenvs/uorc-codedevelopment-AxUFDDNF-py3.12/bin:/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


# SRA IDs identfied - download and process FASTQ files

The steps here (in the bash script, I don't see any LLM needed):
- prefetch (ID)
- fasterq-dump (ID)
- head -n (whatever)
- sort
- gzip

Unfortunately, the download is a FASTQ file rather than a zipped file which is somewhat inconvenient (from a space perspective)

In [7]:
pwd

/home/myuser/work/notebooks/2_extract_data


In [8]:
cd ~/work/results/2_extract_data/scratch

In [9]:
prefetch SRR29101291

2024-08-15T00:52:41 prefetch.3.1.1: 1) Resolving 'SRR29101291'...
2024-08-15T00:52:43 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-08-15T00:52:45 prefetch.3.1.1: 1) Downloading 'SRR29101291'...
2024-08-15T00:52:45 prefetch.3.1.1:  SRA Normalized Format file is being retrieved
2024-08-15T00:52:45 prefetch.3.1.1:  Downloading via HTTPS...
2024-08-15T01:01:04 prefetch.3.1.1:  HTTPS download succeed
2024-08-15T01:01:34 prefetch.3.1.1:  'SRR29101291' is valid: 3567850922 bytes were streamed from 3567845799
2024-08-15T01:01:34 prefetch.3.1.1: 1) 'SRR29101291' was downloaded successfully
2024-08-15T01:01:34 prefetch.3.1.1: 'SRR29101291' has 0 unresolved dependencies


In [14]:
cd ~/work/notebooks/2_extract_data

In [17]:
git clone https://github.com/enasequence/enaBrowserTools.git

Cloning into 'enaBrowserTools'...
remote: Enumerating objects: 845, done.        
remote: Counting objects: 100% (117/117), done.        
remote: Compressing objects: 100% (67/67), done.        
remote: Total 845 (delta 54), reused 63 (delta 41), pack-reused 728 (from 1)        
Receiving objects: 100% (845/845), 337.95 KiB | 1.98 MiB/s, done.
Resolving deltas: 100% (459/459), done.


In [18]:
alias enaDataGet=$PWD/enaBrowserTools/python3/enaDataGet
alias enaGroupGet=$PWD/enaBrowserTools/python3/enaGroupGet

In [21]:
enaDataGet

Traceback (most recent call last):
  File "/home/myuser/work/notebooks/2_extract_data/enaBrowserTools/python3/enaDataGet.py", line 23, in <module>
    import sequenceGet
  File "/home/myuser/work/notebooks/2_extract_data/enaBrowserTools/python3/sequenceGet.py", line 22, in <module>
    import utils
  File "/home/myuser/work/notebooks/2_extract_data/enaBrowserTools/python3/utils.py", line 32, in <module>
    from configparser import SafeConfigParser
ImportError: cannot import name 'SafeConfigParser' from 'configparser' (/usr/local/lib/python3.12/configparser.py). Did you mean: 'RawConfigParser'?


: 1

In [1]:
pwd

/home/myuser/work/notebooks/2_extract_data


In [ ]:
./download_FASTQs.sh -i ./results.txt -n 80000 -o ../../results/2_extract_data/scratch/FASTQs_GSE268034

Processing SRR29101291...


The above works fantastically - limiting the number of "spots" which are read makes this incredibly quick (< 10 minutes). 

# 30 October Update

Ok I have an updated script, let's run it (because I can't run it in the terminal)

In [7]:
./process_geo_accession.sh -g GSE268034 -n 8000 -o ./GSE268034_data

./process_geo_accession.sh: line 42: geo_accession: unbound variable


: 1

Nice. The above looks like it works. I'm not sure what sort of other test cases I would need to do to make sure this is robust? 

(For when I return from lunch) What I will want to do - probably change the script to remove some of the... back up results and error output (this was needed for some debugging)

(This is done)

This script is nice - it will take in a GEO accession and get me the FASTQ files. I don't think much decision making will be needed here? At best, what I would need is to perhaps specify the output directory. However, as an independent workflow, I am calling this script complete.

In [12]:
./process_geo.sh --geo_accession GSE268034 --output_dir ./GSE268034_data --num_spots 80000 --force

2024-10-31 06:49:09 - Processing GEO accession: GSE268034
2024-10-31 06:49:09 - Output directory: ./GSE268034_data
2024-10-31 06:49:09 - Force overwrite is enabled.
2024-10-31 06:49:09 - Number of spots to download: 80000
2024-10-31 06:49:09 - Starting metadata download using download_metadata.R
2024-10-31 06:49:09 - Executing: Rscript download_metadata.R --geo_accession 'GSE268034' --output_dir './GSE268034_data'
Using existing output directory: ./GSE268034_data
Retrieving metadata for GEO accession: GSE268034
Saving metadata to: ./GSE268034_data/GSE268034_series_matrix_metadata.csv
Metadata downloaded and saved successfully!
2024-10-31 06:49:32 - Metadata download completed.
2024-10-31 06:49:32 - Starting FASTQ download using download_fastqs.sh
2024-10-31 06:49:32 - Executing: ./download_fastqs.sh --geo_accession 'GSE268034' --output_dir './GSE268034_data' --num_spots 80000 --force
Read 80000 spots for SRR29101291
Written 80000 spots for SRR29101291
Read 80000 spots for SRR29101292
W